# 文本预处理

In [1]:
import collections
import torch
import re

## 将数据集提取并简单处理
从H.G.Well的时光机器中加载文本。 这是一个相当小的语料库，只有30000多个单词， 而现实中的文档集合可能会包含数十亿个单词。 下面的函数将数据集读取到由多条文本行组成的列表中，其中每条文本行都是一个字符串。 在这里忽略了标点符号和字母大写。

In [5]:
with open("./data_set/H_G_Well_time_machine.txt", "r") as f:
    lines = f.readlines()
# 忽略标点符号和字母大写
out = []
for line in lines:
    # ^取非 +连续选取多个匹配
    line = re.sub('[^A-Za-z]+', ' ', line).strip().lower()
    if line:
        out.append(line)
lines = out
del(out)

## 词元化
将文本序列拆分为词元(token), 将一个句子转化为词元序列。

In [211]:
def tokenize(lines, token='word'):
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(5):
    print(tokens[i])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells']
['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and']
['most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions']
['whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms']
['of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at']


## 词表
词元的类型是字符串，而模型需要的输入是数字。构建一个字典，通常也叫做词表（vocabulary）， 用来将字符串类型的词元映射到从开始的数字索引中。
-  先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计， 得到的统计结果称之为语料（corpus）。 然后根据每个唯一词元的出现频率，为其分配一个数字索引。 很少出现的词元通常被移除，这可以降低复杂性。 另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“\<unk\>”。 我们可以选择增加一个列表，用于保存那些被保留的词元， 例如：填充词元（“\<pad\>”）； 序列开始词元（“\<bos\>”）； 序列结束词元（“\<eos\>”）。

In [224]:
class Vocab:
    """一个词元字典类的实现"""
    def __init__(self, tokens:list, min_freq=0, reserved_tokens:list=None) -> None:
        if tokens is not None:
            # 当第一个条件满足时，就不会跳到第二个判断，避免了空列表报错的情况。
            if len(tokens)!=0 and isinstance(tokens[0], list):
                tokens = [i for line in tokens for i in line]
        else:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter=collections.Counter(tokens)
        # 按出现词频从高到低排序
        self._token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True)
        # 通过列表,利用序号访问词元。
        self.idx_to_token = ['<unk>'] + reserved_tokens # 未知词元<unk>的索引为0, 保留词元排在最前
        self.token_to_idx = {
            i: k
            for k, i in enumerate(self.idx_to_token) 
        }
        
        for token, freq in self._token_freqs:
            if freq < min_freq:  # 过滤掉出现频率低于要求的词
                break
            if token not in self.token_to_idx:  
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
        
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, input_tokens):
        """输入单字串或序列, 将其全部转化为序号编码"""
        if isinstance(input_tokens, str):
            return self.token_to_idx.get(input_tokens, 0)
        return [self.__getitem__(token) for token in input_tokens]
    
    def __repr__(self) -> str:
        show_items = 5 if len(self) > 5 else len(self)
        out = f"<Vocab with {len(self)} tokens: "
        for i in range(show_items):
            out += f'"{self.idx_to_token[i]}", '
        out += "...>"
        return out

    def to_tokens(self, input_keys):
        """输入单s索引或序列, 将其全部转化为词元"""
        if isinstance(input_keys, int):
            return self.idx_to_token[input_keys] if input_keys < len(self) else self.idx_to_token[0]
        elif isinstance(input_keys, (list, tuple)):
            return [self.to_tokens(keys) for keys in input_keys]
        else:
            return self.idx_to_token[0]



# 整合
函数返回corpus（词元索引列表）和vocab（时光机器语料库的词表）。使用字符（而不是单词）实现文本词元化。

In [ ]:
tokens = tokenize(lines, "char")
vocab = Vocab(tokens)
corpus = [vocab[token] for line in tokens for token in line]
len(corpus), vocab